## Analysis of Heroes of Pymoli Data
Completed March 2019

### Observable Trends

Key trends observed in the data are as follows:

- Of the total revenue (USD 2,379.77), the vast majority is generated from male players (USD 1,967.64). However, female players on average purchase 10% more by value than male players (respectively USD 4.47 vs USD 4.07 average total purchase per person).
- Close to half of our revenue (USD 1,114.06) is generated from our peak age demographic of 20-24. However, players between the age of 35-39 and <10 spend on average USD 4.76 and USD 4.54, respectivly per person. This is 10% (USD 0.44) and 5% (USD 0.22) higher than the USD 4.32 average total purchase per person of the 20-24 year old players.
- The top selling item, both by value (USD 50.76) and by count (12) is Oathbreaker, Last Hope of the Breaking Storm

In [1]:
# Setup
import pandas as pd
import numpy as np

# Load file
file = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
data = pd.read_csv(file)

data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


### Player Count

In [2]:
# 1.0 To determine 'Total Number of Players'

## 1.1 Use Pandas value_count function to count the SN's
countSN = data["SN"].value_counts()

## 1.2 Create DataFrame for 'Total Players' using the length of the count
PlayerTotal = pd.DataFrame({"Total Players": [len(countSN)]})

## 1.3 Print Results
PlayerTotal

,Total Players
0,576


### Purchasing Analysis (Total)

In [3]:
# 2.0   Create a DataFrame analyzing total results
    
## 2.1  To determine 'Number of Unique Items' use Pandas value_count
##      function to count 'Item ID'
countItem = data["Item ID"].value_counts()

## 2.2  To determine 'Average Purchase Price' use Pandas mean
##      function to calculate the average of 'Price'
averagePrice = data["Price"].mean()

## 2.3  To determine 'Total Number of Purchases' use Pandas value_count
##      Pandas count function to calculate the average of 'Purchase ID'
countPurchaseID = data["Purchase ID"].value_counts()

## 2.4  To determine 'Total Revenue' use Pandas sum
##      function to calculate the average of 'Price'
totalPrice = data["Price"].sum()

## 2.5 Combine results into DataFrame for 'Purchasing Analysis (Total)'
AnalysisTotal = pd.DataFrame({
    "Number of Unique Items": [len(countItem)],
    "Average Price": [averagePrice],
    "Number of Purchases": [len(countPurchaseID)],
    "Total Revenue": [totalPrice]})

## 2.6 Clean Data - convert 'Average Price' and 'Total Revenue' to currency format
AnalysisTotal["Average Price"] = AnalysisTotal["Average Price"].map("$ {:.2f}".format)
AnalysisTotal["Total Revenue"] = AnalysisTotal["Total Revenue"].map("$ {:.2f}".format)

## 2.7 Print DataFrame
AnalysisTotal


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$ 3.05,780,$ 2379.77


### Gender Demographics

In [4]:
# 3.0   Create a DataFrame analyzing gender demographics


## 3.1 Clean Data

## 3.1.1 Clean Data - drop duplicate SN numbers
data.drop_duplicates(subset="SN", keep='first', inplace=True)

## 3.1.2 Clean Data - filter for only columns 'SN' and 'Gender'
genderdata_0 = data[["Gender","SN"]]

## 3.1.3 Clean Data - rename columns
genderdata_1 = genderdata_0.rename(columns={"Gender": "Gender", "SN":"Total Count"})

## 3.1.4 Clean Data - group DataFrame to index by 'Gender'
genderdata_2 = genderdata_1.groupby(["Gender"])

## 3.1.5 Clean Data - group DataFrame to count 'SN' column
genderdata_3 = genderdata_2.count()

## 3.1.6 Clean Data - rename columns
genderdata_4 = genderdata_3.rename(columns={"Gender": "", "Total Count":"Total Count"})

## 3.1.7 Clean Data - sort by 'Total Count'
genderdata = genderdata_4.sort_values("Total Count", ascending=False)


## 3.2 Calculate and Append column

## 3.2.1 Define calculation for 'Percentage of Players' column
percentage_of_players = genderdata["Total Count"]/genderdata["Total Count"].sum()*100

## 3.2.2 Append calculated column to DataFrame
genderdata["Percentage of Players"] = percentage_of_players.map("{:.2f} %".format)

## 3.3 Print DataFrame
genderdata


## NOTES: Steps to Clean Data (ie 3.1.1 to 3.1.6) can be combined to 1 command as follows:
##        genderdata = data[["Gender","SN"]].groupby(["Gender"]).count().set_index().rename(columns={"Gender":"", "SN":"Total Count"}).sort_values("Total Count", ascending=False)


,Total Count,Percentage of Players
Gender,,
Male,484,84.03 %
Female,81,14.06 %
Other / Non-Disclosed,11,1.91 %


### Purchasing Analysis (Gender)

In [5]:
# 4.0   Create a DataFrame analyzing purchasing by gender

## 4.0.1 reset data file (duplicates were dropped above)
data_A = pd.read_csv(file)


# 4.1  Clean Data

## 4.1.1 Clean Data - filter for only columns 'Price' and 'Gender'
Gen_Purchasedata_0 = data_A[["Gender", "Price"]]

## 4.1.2 Clean Data - group by gender
Gen_Purchasedata_1 = Gen_Purchasedata_0.groupby(["Gender"])

## 4.1.3 Clean Data - summarise price by count, sum and mean based on gender groupings
Gen_Purchasedata_count = Gen_Purchasedata_1.count()
Gen_Purchasedata_mean = Gen_Purchasedata_1.mean()
Gen_Purchasedata_sum = Gen_Purchasedata_1.sum()

## 4.1.4 Clean Data - combine into new dataframe
Gen_Purchasedata = pd.DataFrame({"Total Count": Gen_Purchasedata_count["Price"],
                                   "Average Purchase Price": Gen_Purchasedata_mean["Price"].map("$ {:.2f}".format),
                                   "Total Purchase Value": Gen_Purchasedata_sum["Price"].map("$ {:,.2f}".format)})

# 4.2  Calculations

## 4.2.1 Define calculation for 'Avg Total Purchase per Person' column
avg_total_purchase_per_person = Gen_Purchasedata_sum["Price"] / genderdata_3["Total Count"]

## 4.2.2 Append calculated column to DataFrame
Gen_Purchasedata["Avg Total Purchase per Person"] = avg_total_purchase_per_person.map("$ {:.2f}".format)

# 4.3 Print DataFrame
Gen_Purchasedata


,Total Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$ 3.20,$ 361.94,$ 4.47
Male,652,$ 3.02,"$ 1,967.64",$ 4.07
Other / Non-Disclosed,15,$ 3.35,$ 50.19,$ 4.56


### Age Demographics

In [6]:
# 5.0   Create a DataFrame analyzing users by age

# create bins and group_names
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
group_names = [">10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# append 'Age Bracket' column to data
data["Age Bracket"] = pd.cut(data["Age"], bins, labels=group_names)

# Filter for only columns 'SN' and 'Age Bracket'
age_data_0 = data[["Age Bracket", "SN"]]

# Group data by age bracket
age_data_1 = age_data_0.groupby(["Age Bracket"]).count()

# Rename Columns
age_data = age_data_1.rename(columns={"Age Bracket": "", "SN":"Total Count"})

# Define Percentage of Players calculation
percentage_of_players = age_data["Total Count"] / age_data["Total Count"].sum()*100

## Append calculated column to DataFrame
age_data["Percentage of Players"] = percentage_of_players.map("{:.2f} %".format)

# Print DataFrame
age_data


,Total Count,Percentage of Players
Age Bracket,,
>10,17,2.95 %
10-14,22,3.82 %
15-19,107,18.58 %
20-24,258,44.79 %
25-29,77,13.37 %
30-34,52,9.03 %
35-39,31,5.38 %
40+,12,2.08 %


### Purchasing Analysis (Age)

In [7]:
# 6.0   Create a DataFrame analyzing purchasing by age


# 6.1  Clean Data

## 6.1.1 Clean Data - append age bracket grouping
data_A["Age Bracket"] = pd.cut(data_A["Age"], bins, labels=group_names)

## 6.1.2 Clean Data - filter for only columns 'Age Bracket' and 'Price'
age_purch_data_0 = data_A[["Age Bracket", "Price"]]

## 6.1.3 Clean Data - group by 'Age Bracket'
age_purch_data_1 = age_purch_data_0.groupby(["Age Bracket"])

## 6.1.4 Clean Data - summarise price by count, sum and mean based on age bracket groupings
age_purch_data_count = age_purch_data_1.count()
age_purch_data_mean = age_purch_data_1.mean()
age_purch_data_sum = age_purch_data_1.sum()

## 6.1.5 Clean Data - combine into new dataframe
age_purch_data = pd.DataFrame({"Purchase Count": age_purch_data_count["Price"],
                                   "Average Purchase Price": age_purch_data_mean["Price"].map("$ {:.2f}".format),
                                   "Total Purchase Value": age_purch_data_sum["Price"].map("$ {:,.2f}".format)})

# 6.2  Calculations

## 6.2.1 Define calculation for 'Avg Total Purchase per Person' column
avg_total_purchase_per_person_2 = age_purch_data_sum["Price"] / age_data["Total Count"]

## 6.2.2 Append calculated column to DataFrame
age_purch_data["Avg Total Purchase per Person"] = avg_total_purchase_per_person_2.map("$ {:.2f}".format)



# 6.3 Print DataFrame
age_purch_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bracket,,,,
>10,23,$ 3.35,$ 77.13,$ 4.54
10-14,28,$ 2.96,$ 82.78,$ 3.76
15-19,136,$ 3.04,$ 412.89,$ 3.86
20-24,365,$ 3.05,"$ 1,114.06",$ 4.32
25-29,101,$ 2.90,$ 293.00,$ 3.81
30-34,73,$ 2.93,$ 214.00,$ 4.12
35-39,41,$ 3.60,$ 147.67,$ 4.76
40+,13,$ 2.94,$ 38.24,$ 3.19


### Top Spenders

In [8]:
# 7.0   Create a DataFrame analyzing top spenders


# 7.1  Clean Data

## 7.1.1 Clean Data - filter for only columns 'SN' and 'Price'
top_spend_data_0 = data_A[["SN", "Price"]]

## 7.1.2 Clean Data - group by individual
top_spend_data_1 = top_spend_data_0.groupby(["SN"])

## 7.1.3 Clean Data - summarise price by count, sum and mean based on SN groupings
top_spend_data_count = top_spend_data_1.count()
top_spend_data_mean = top_spend_data_1.mean()
top_spend_data_sum = top_spend_data_1.sum()

## 7.1.4 Clean Data - combine into new dataframe
top_spend_data = pd.DataFrame({"Purchase Count": top_spend_data_count["Price"],
                                   "Average Purchase Price": top_spend_data_mean["Price"].map("$ {:.2f}".format),
                                   "Total Purchase Value": top_spend_data_sum["Price"]})

# 7.1.5 Clean Data - Sort by Total Purchase Value
top_spend = top_spend_data.sort_values(["Total Purchase Value"], ascending=False)

# 7.1.6 Clean Data - Format 'Total Purchase Value' values
top_spend["Total Purchase Value"] = top_spend["Total Purchase Value"].map("$ {:,.2f}".format)


# 7.2 print dataframe
top_spend.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$ 3.79,$ 18.96
Idastidru52,4,$ 3.86,$ 15.45
Chamjask73,3,$ 4.61,$ 13.83
Iral74,4,$ 3.40,$ 13.62
Iskadarya95,3,$ 4.37,$ 13.10


### Most Popular Items

In [9]:
# 8.0   Create a DataFrame analyzing top items based on purchase value


# 8.1  Clean Data

## 8.1.1 Clean Data - filter for only columns 'Item ID', 'Item Description' and 'Price'
item_data_0 = data_A[["Item ID", "Item Name", "Price"]]

## 8.1.2 Clean Data - group by individual
item_data_1 = item_data_0.groupby(["Item ID", "Item Name"])

## 8.1.3 Clean Data - summarise price by count, sum and mean based on Item groupings
item_data_count = item_data_1.count()
item_data_mean = item_data_1.mean()
item_data_sum = item_data_1.sum()

## 8.1.4 Clean Data - combine into new dataframe
item_data = pd.DataFrame({"Purchase Count": item_data_count["Price"],
                                   "Item Price": item_data_mean["Price"],
                                   "Total Purchase Value": item_data_sum["Price"]})

# 8.1.5 Clean Data - Sort by Total Purchase Value
top_item_spend = item_data.sort_values(["Purchase Count"], ascending=False)

# 8.1.6 Clean Data - Format 'Total Purchase Value' and 'Item Price' values
top_item_spend["Total Purchase Value"] = top_item_spend["Total Purchase Value"].map("$ {:,.2f}".format)
top_item_spend["Item Price"] = top_item_spend["Item Price"].map("$ {:,.2f}".format)

# 8.2 print dataframe
top_item_spend.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 4.23,$ 50.76
145,Fiery Glass Crusader,9,$ 4.58,$ 41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$ 3.53,$ 31.77
82,Nirvana,9,$ 4.90,$ 44.10
19,"Pursuit, Cudgel of Necromancy",8,$ 1.02,$ 8.16


### Most Profitable Items

In [10]:
# 9.0   Create a DataFrame analyzing top items based on item price

# 9.1 Clean Data - Sort by Total Purchase Value
top_item_price = item_data.sort_values(["Total Purchase Value"], ascending=False)

# 9.2 Clean Data - Format 'Total Purchase Value' and 'Item Price' values
top_item_price["Total Purchase Value"] = top_item_price["Total Purchase Value"].map("$ {:,.2f}".format)
top_item_price["Item Price"] = top_item_price["Item Price"].map("$ {:,.2f}".format)

# 9.3 print dataframe
top_item_price.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 4.23,$ 50.76
82,Nirvana,9,$ 4.90,$ 44.10
145,Fiery Glass Crusader,9,$ 4.58,$ 41.22
92,Final Critic,8,$ 4.88,$ 39.04
103,Singed Scalpel,8,$ 4.35,$ 34.80
